# SQL Agent

Expanding on our previous agents notebook, we will now look at using an agent to query a SQL database.

In [19]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

import sys

if ".." not in sys.path:
    sys.path.insert(0, "..")

from utils import load_envrc

load_envrc("../../.envrc")

## Dataset: North Carolina General Statutes

We will use the North Carolina General Statutes as our dataset. The statutes are available online at https://www.ncleg.gov/Laws/GeneralStatutesTOC. The `download-nc-statutes.py` script downloads the statutes and saves them to a CSV file.

In [20]:
import pandas as pd

df = pd.read_csv("input/nc_general_statutes.csv.gz")
df.head(5)

,chapter_number,chapter_title,article_number,article,section_number,text,source_url
0,1,Civil Procedure,1,Definitions,1-1,§ 1-1. Remedies.\nRemedies in the courts of j...,https://www.ncleg.gov/EnactedLegislation/Statu...
1,1,Civil Procedure,1,Definitions,1-2,§ 1-2. Actions.\nAn action is an ordinary pro...,https://www.ncleg.gov/EnactedLegislation/Statu...
2,1,Civil Procedure,1,Definitions,1-3,§ 1-3. Special proceedings.\nEvery other reme...,https://www.ncleg.gov/EnactedLegislation/Statu...
3,1,Civil Procedure,1,Definitions,1-4,§ 1-4. Kinds of actions.\nActions are of two ...,https://www.ncleg.gov/EnactedLegislation/Statu...
4,1,Civil Procedure,1,Definitions,1-5,§ 1-5. Criminal action.\nA criminal action is...,https://www.ncleg.gov/EnactedLegislation/Statu...


In [21]:
# Let's look at a specific statute to see what the data looks like.

print(df[df["section_number"] == "15A-145"].iloc[0]["text"])

§ 15A-145.  Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; expunction of certain other misdemeanors.
(a)	Whenever any person who has not previously been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to G.S. 18B-302(b)(1), and the offense was committed before the person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years after the date of the conviction, or (ii) the completion of any period of probation, whichever occur

In [22]:
# Now let's load the data into a PostgreSQL database.

import os
from sqlalchemy import create_engine

pg_engine = create_engine(os.getenv("DATABASE_URL"))

df.to_sql("nc_general_statutes", pg_engine, if_exists="replace", index=False)

-1

### Query the data

Let's think about different ways we can query the data. Some simple examples include:

- "How many chapters are there in total?"
- "What is the title of Chapter 15A?"
- "List all statutes in Chapter 15A."
- "What is the text of statute 15A-145?"

We can obviously ask more complex questions, but these simple ones will do for now. Let's try to answer these questions using SQL queries.

In [23]:
# "How many chapters are there in total?"

pd.read_sql("SELECT COUNT(DISTINCT chapter_number) FROM nc_general_statutes", pg_engine)

,count
0,395


In [24]:
# "What is the title of Chapter 15A?"

pd.read_sql(
    "SELECT DISTINCT chapter_title FROM nc_general_statutes WHERE chapter_number = '15A'", pg_engine
)

,chapter_title
0,Criminal Procedure Act


In [25]:
# "List all articles in Chapter 15A."

pd.read_sql(
    """
    SELECT DISTINCT 
        CAST(REGEXP_REPLACE(article_number, '[^0-9]', '', 'g') AS INTEGER) AS article_number_sort
        , article_number
        , article FROM nc_general_statutes
    WHERE chapter_number = '15A'
    """,
    pg_engine,
    dtype={"article_number_sort": "Int64"},
).head(20)

,article_number_sort,article_number,article
0,1,1,Definitions and General Provisions
1,2,2,Jurisdiction
2,3,3,Venue
3,4,4,Entry and Withdrawal of Attorney in Criminal Case
4,5,5,Expunction of Records
5,6,6,Certificate of Relief
6,7,7,Certificate of Relief
7,8,8,Electronic Recording of Interrogations
8,8,8A,SBI and State Crime Laboratory Access to View ...
9,9,9,Search and Seizure by Consent


# NC Statute SQL Agent

Now let's create a SQL agent that can answer questions about the North Carolina General Statutes. In addition to the questions above, ideally the agent can take certain questions further, for example: "Explain like I'm 5: Statute 15A-145".